<a href="https://colab.research.google.com/github/parakram93/ml_foundations/blob/main/rnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn

class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size,num_layers=3, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, h = self.rnn(x)
        out = self.fc(out)
        return out, h

# Dummy input: 1 batch, 3 words, 3 features per word
sentence = torch.tensor([
    [ [1.0, 0.0, 0.0],
      [0.0, 1.0, 0.0],
      [0.5, 0.5, 0.0] ]
], dtype=torch.float)   # always good to specify float


print("Input shape :", sentence.shape)

model = SimpleRNN(input_size=3, hidden_size=5, output_size=2)
outputs, hidden = model.forward(sentence)

print("Output shape :", outputs.shape)
print("Hidden shape :", hidden.shape)


Input shape : torch.Size([1, 3, 3])
Output shape : torch.Size([1, 3, 2])
Hidden shape : torch.Size([3, 1, 5])


In [ ]:
import pandas as pd

df = pd.read_csv('100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [ ]:
#tokenize
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'",'')
  return text.split()

tokenize("hello world?")


['hello', 'world']

In [ ]:
#vocab
vocab = {'<UNK>':0}


In [ ]:
def build_vocab(row):
  tokenized_question = tokenize(row['question'])
  tokenized_answer = tokenize(row['answer'])

  merged_tokens = tokenized_question + tokenized_answer
  for token in merged_tokens:

    if token not in vocab:
      vocab[token] = len(vocab)


In [ ]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [ ]:
# convert words to numerical indices
def text_to_indices(text, vocab):

  indexed_text = []

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [ ]:
text_to_indices("Hello world", vocab)


[0, 45]

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader

In [ ]:
class QADataset(Dataset):

  def __init__(self,df,vocab):
    self.df = df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self, index):
    numerical_question = text_to_indices(self.df.iloc[index]['question'], self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'], self.vocab)
    return torch.tensor(numerical_question), torch.tensor(numerical_answer)




In [ ]:
dataset = QADataset(df, vocab)

In [ ]:
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

In [ ]:
for question, answer in dataloader:
  print(question, answer)

tensor([[  1,   2,   3, 146, 147,  19, 148]]) tensor([[149]])
tensor([[ 1,  2,  3,  4,  5, 53]]) tensor([[54]])
tensor([[  1,   2,   3, 180, 181, 182, 183]]) tensor([[184]])
tensor([[  1,   2,   3,   4,   5, 135]]) tensor([[136]])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([[36]])
tensor([[42, 43, 44, 45, 46, 47, 48]]) tensor([[49]])
tensor([[ 42, 101,   2,   3,  17]]) tensor([[102]])
tensor([[ 42, 312,   2, 313,  62,  63,   3, 314, 315]]) tensor([[316]])
tensor([[ 1,  2,  3, 92, 93, 94]]) tensor([[95]])
tensor([[ 42, 216, 118, 217, 218,  19,  14, 219,  43]]) tensor([[220]])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([[74]])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([[68]])
tensor([[ 42,  18,   2,   3, 281,  12,   3, 282]]) tensor([[205]])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([[58]])
tensor([[  1,   2,   3, 103,   5, 104,  19, 105]]) tensor([[106]])
tensor([[ 42,  86,  87, 241, 242,  19,  39, 243]]) tensor([[244]])
tensor([[ 10,  75,   3, 296,  19, 297]]) tensor([[29

In [ ]:
import torch.nn as nn

In [ ]:
class SimpleRNN(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, embedding_dim=50)
    self.rnn = nn.RNN(50, 64, batch_first=True)
    self.fc = nn.Linear(64, vocab_size)

  def forward(self, question):
    embedded_question = self.embedding(question)
    hidden, final = self.rnn(embedded_question)
    output = self.fc(final.squeeze(0))

    return output

In [ ]:
y = nn.Embedding(324, embedding_dim=50)
z =dataset[10][0]
b = y(z)
x = nn.RNN(50,64,batch_first=True)
c,d = x(b)
print(c.shape)
print(d.shape)



torch.Size([6, 64])
torch.Size([1, 64])


In [ ]:
learning_rate = 0.001
epochs = 20

In [ ]:
model = SimpleRNN(len(vocab))
a_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


In [ ]:
for epoch in range(epochs):
  total_loss = 0

  for question, answer in dataloader:


    optimizer.zero_grad()

    output = model.forward(question)

    loss = a_loss(output,answer[0])
    loss.backward()

    optimizer.step()

    total_loss += loss.item()

  print(f"Epoch: {epoch+1}, Loss: {total_loss:4f}")

Epoch: 1, Loss: 525.842284
Epoch: 2, Loss: 457.881875
Epoch: 3, Loss: 376.598355
Epoch: 4, Loss: 315.051475
Epoch: 5, Loss: 263.519734
Epoch: 6, Loss: 215.883261
Epoch: 7, Loss: 171.027251
Epoch: 8, Loss: 132.660964
Epoch: 9, Loss: 102.131030
Epoch: 10, Loss: 77.890881
Epoch: 11, Loss: 61.028518
Epoch: 12, Loss: 46.996479
Epoch: 13, Loss: 37.436921
Epoch: 14, Loss: 30.527876
Epoch: 15, Loss: 24.958444
Epoch: 16, Loss: 20.781418
Epoch: 17, Loss: 17.620165
Epoch: 18, Loss: 14.998881
Epoch: 19, Loss: 13.090411
Epoch: 20, Loss: 11.259684


In [ ]:
def predict(model,question,threshold = 0.5):
  numerical_question = text_to_indices(question, vocab)
  question_tensor = torch.tensor(numerical_question).unsqueeze(0)
  output = model(question_tensor)
  probs = torch.nn.functional.softmax(output, dim=1)
  value, index = torch.max(probs, dim=1)
  if value < threshold:
    print("I don't know")
  print(list(vocab.keys())[index])

In [ ]:
predict(model,"What id your name?")

I don't know
batman


In [3]:
!pip install numpy<2


/bin/bash: line 1: 2: No such file or directory


In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

# -------------------------
# 1. Prepare Data
# -------------------------
data = "hello world"
chars = sorted(list(set(data)))
vocab_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

def encode(s):
    return [stoi[c] for c in data]

def decode(idxs):
    return "".join(itos[i] for i in idxs)

encoded = encode(data)
input_ids  = torch.tensor(encoded[:-1])
target_ids = torch.tensor(encoded[1:])   # next-char prediction


# -------------------------
# 2. Define RNN Model (PyTorch built-in)
# -------------------------
class CharRNN(nn.Module):
    def __init__(self, vocab_size, emb_size=32, hidden_size=64):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size)
        self.rnn = nn.RNN(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)

    def forward(self, x, h):
        x = self.embedding(x)          # (B, T, emb)
        out, h = self.rnn(x, h)        # (B, T, hidden)
        logits = self.fc(out)          # (B, T, vocab)
        return logits, h

    def init_hidden(self):
        return torch.zeros(1, 1, 64)   # (num_layers, batch, hidden)


model = CharRNN(vocab_size)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


# -------------------------
# 3. Training
# -------------------------
num_epochs = 500

for epoch in range(num_epochs):
    optimizer.zero_grad()

    h = model.init_hidden()
    x = input_ids.unsqueeze(0)      # (1, T)
    y = target_ids.unsqueeze(0)     # (1, T)

    logits, h = model(x, h)
    loss = criterion(logits.view(-1, vocab_size), y.view(-1))

    loss.backward()
    optimizer.step()

    if epoch % 100 == 0:
        print(f"Epoch {epoch} | Loss = {loss.item():.4f}")


# -------------------------
# 4. Sampling / Text Generation
# -------------------------
def sample(start_char='h', length=40):
    model.eval()
    idx = torch.tensor([[stoi[start_char]]])
    h = model.init_hidden()

    output_string = start_char

    for _ in range(length):
        logits, h = model(idx, h)
        probs = torch.softmax(logits[0, -1], dim=0).detach().numpy()
        next_idx = np.random.choice(vocab_size, p=probs)

        output_string += itos[next_idx]
        idx = torch.tensor([[next_idx]])

    return output_string


print("\nGenerated text:")
print(sample("h"))


Epoch 0 | Loss = 2.0790
Epoch 100 | Loss = 0.0003
Epoch 200 | Loss = 0.0002
Epoch 300 | Loss = 0.0001
Epoch 400 | Loss = 0.0001

Generated text:
hello worldo worldo worldo worldo worldo 


In [ ]:
pip install torchtext==0.16.2


In [4]:
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset

# Sample dataset
sentences = [
    "I love this movie",
    "This film is great",
    "I hate this movie",
    "This  is terrible"
]

labels = [1, 1, 0, 0]

# 1. Tokenize sentences (split words)
def tokenize(sentence):
    return sentence.lower().split()

tokenized_sentences = [tokenize(s) for s in sentences]

# 2. Build vocabulary
vocab = build_vocab_from_iterator(tokenized_sentences, specials=["<pad>"])  #goes through all tokenized sentences and assigns unique integer IDs to each word. specials=["<pad>"] → adds a padding token with index 0
vocab.set_default_index(vocab["<pad>"])  # unknown words → pad index

# 3. Encode sentences
encoded_sentences = [torch.tensor(vocab(s)) for s in tokenized_sentences] #Each word is replaced by its integer index using vocab(word_list), ["i", "love", "this", "movie"] → [1, 2, 3, 4]
vocab_size = len(vocab)
# 4. Pad sequences to same length
#pad_sequence ensures all sentences have the same length, which is required to form a batch
#batch_first=True → output shape will be (batch_size, max_len) instead of (max_len, batch_size)
#padding_value=vocab["<pad>"] → fills shorter sentences with 0 (padding token)
padded_sentences = pad_sequence(encoded_sentences, batch_first=True, padding_value=vocab["<pad>"])
print("Padded Sentences:\n", padded_sentences)

# Output: tensor of shape (batch_size, max_len)

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, emb_size=32, hidden_size=64, output_size=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb_size, padding_idx=0)
        self.rnn = nn.RNN(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), 64)  # initial hidden state
        x = self.embedding(x)               # (B, T, emb)
        out, h = self.rnn(x, h0)           # out=(B, T, H)
        out = out[:, -1, :]                 # take last hidden state
        out = self.fc(out)                  # (B, output_size)
        return out

model = SentimentRNN(vocab_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

num_epochs = 200
targets = torch.tensor(labels)
for epoch in range(num_epochs):
    optimizer.zero_grad()
    logits = model(padded_sentences)
    loss = criterion(logits, targets)
    loss.backward()
    optimizer.step()

    if epoch % 50 == 0:
        print(f"Epoch {epoch} | Loss = {loss.item():.4f}")



Padded Sentences:
 tensor([[2, 8, 1, 4],
        [1, 5, 3, 6],
        [2, 7, 1, 4],
        [1, 3, 9, 0]])
Epoch 0 | Loss = 0.6723
Epoch 50 | Loss = 0.0000
Epoch 100 | Loss = 0.0000
Epoch 150 | Loss = 0.0000


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from torchtext.vocab import build_vocab_from_iterator

# --------------------------------------------------
# 1. Load IMDb small subset for safety
# --------------------------------------------------
data = load_dataset("imdb")
train_texts = data["train"]["text"][:5000]   # take 5k for Colab safety
train_labels = torch.tensor(data["train"]["label"][:5000])

MAX_LEN = 200  # limit review length


# --------------------------------------------------
# 2. Tokenizer
# --------------------------------------------------
def tokenize(s):
    return s.lower().split()


tokenized = [tokenize(s) for s in train_texts]


# --------------------------------------------------
# 3. Build vocabulary
# --------------------------------------------------
vocab = build_vocab_from_iterator(tokenized, specials=["<pad>", "<unk>"])
vocab.set_default_index(vocab["<unk>"])
PAD_IDX = vocab["<pad>"]


# --------------------------------------------------
# 4. Encode + Truncate + Pad
# --------------------------------------------------
def encode(sentence):
    ids = vocab(sentence)
    ids = ids[:MAX_LEN]              # truncate
    return torch.tensor(ids)


encoded = [encode(s) for s in tokenized]

padded = pad_sequence(
    encoded,
    batch_first=True,
    padding_value=PAD_IDX
)

# ensure shape is (N, MAX_LEN)
if padded.size(1) < MAX_LEN:  #If this is e.g. 180, but MAX_LEN=200, we still need 20 more paddings.
    padded = torch.nn.functional.pad(
        padded,
        (0, MAX_LEN - padded.size(1)),
        value=PAD_IDX
    )


dataset = torch.utils.data.TensorDataset(padded, train_labels) #TensorDataset is a PyTorch utility that allows you to pair tensors together and turn them into a dataset that can be used with a DataLoader
#we use tensor dataset when data is already tensor, numbers and aligned proprerly. else we have to make custom dataset.
loader = DataLoader(dataset, batch_size=32, shuffle=True)


# --------------------------------------------------
# 5. Define RNN model
# --------------------------------------------------
class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, emb=32, hidden=64, out=2):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, emb, padding_idx=PAD_IDX)
        self.rnn = nn.RNN(emb, hidden, batch_first=True)
        self.fc = nn.Linear(hidden, out)

    def forward(self, x):
        h0 = torch.zeros(1, x.size(0), 64)
        x = self.embedding(x)
        out, h = self.rnn(x, h0)
        out = out[:, -1, :]    # last hidden state
        return self.fc(out)


model = SentimentRNN(len(vocab))
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.005)


# --------------------------------------------------
# 6. Training loop
# --------------------------------------------------
for epoch in range(5):     # keep small for demo
    total_loss = 0
    for batch_x, batch_y in loader:
        optimizer.zero_grad()
        logits = model(batch_x)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    print(f"Epoch {epoch} | Loss = {total_loss:.4f}")


Epoch 0 | Loss = 8.3994
Epoch 1 | Loss = 0.2322
Epoch 2 | Loss = 0.0108
Epoch 3 | Loss = 0.0061
Epoch 4 | Loss = 0.0040


In [29]:
from torch.nn.utils.rnn import pad_sequence
import torch

def tokenize(s):
    return s.lower().split()



def encode(sentence):
    ids = vocab(sentence)
    ids = ids[:MAX_LEN]  # truncate
    return torch.tensor(ids)

def preprocess(text):
    tokenized = tokenize(text)
    encoded = encode(tokenized)


    # pad to MAX_LEN
    padded = torch.nn.functional.pad(
        encoded,
        (0, MAX_LEN - encoded.size(0)),
        value=PAD_IDX
    )

    return padded.unsqueeze(0)   # shape (1, MAX_LEN)

# --------------------------
# PREDICTION
# --------------------------

with torch.no_grad():
    review = input("Enter the review: ")

    x = preprocess(review)

    logits = model(x)
    print(logits)

    pred_class = torch.argmax(logits, dim=1).item()

    if pred_class == 1:
     print("POSITIVE")
    else:
     print("NEGATIVE")


#this is a very simple model and hence this will not give correct output. so , lstm is rerquired since rnn cannot remember long term datas


Enter the review: its not okay
tensor([[ 5.5799, -5.1940]])
NEGATIVE


In [1]:
#start of lstm

In [1]:
!pip install nltk

In [2]:
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from collections import Counter
from torch.utils.data import Dataset,DataLoader
from nltk.tokenize import word_tokenize
import nltk



In [3]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [4]:
nltk.download("punkt")
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [5]:
tokens = word_tokenize(document.lower())

In [6]:
# build vocab
vocab = {'<unk>':0}

#Counter(token).keys() #provides info of how many tme same word has arrived in a data. .keys() filters uniques words form the tokens

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token] = len(vocab)

len(vocab)

289

In [7]:
input_sentences = document.split('\n')

In [8]:
def text_to_indces(sentence,vocab):
  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence



In [9]:
input_numerical_sentences = []
for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indces(word_tokenize(sentence.lower()),vocab))

print(len(input_numerical_sentences))


78


In [10]:
training_sequence = []
for sentence in input_numerical_sentences:
  for i in range (1,len(sentence)):
    training_sequence.append(sentence[:i+1])

print(len(training_sequence))





942


In [11]:
len_list = []

for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

62

In [12]:
padded_training_sequence = []

for sequence in training_sequence:
  padded_training_sequence.append([0]*(max(len_list)-len(sequence))+sequence)



In [13]:
padded_training_sequence = torch.tensor(padded_training_sequence, dtype=torch.long)

In [14]:
X = padded_training_sequence[:, :-1] #all rows, all columns except last one

y = padded_training_sequence[:, -1] #all rows and only last column

print(X.shape)
print(y.shape)

torch.Size([942, 61])
torch.Size([942])


In [15]:
class CustomDataset(Dataset):

  def __init__(self, X, y):
    self.X = X
    self.y = y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

In [16]:
dataset = CustomDataset(X, y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [17]:
class LSTMModel(nn.Module):

  def __init__(self, vocab_size):
    super().__init__()
    self.embedding = nn.Embedding(vocab_size, 100)
    self.lstm = nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self, x):
    embedded = self.embedding(x)
    intermediate_hidden_states, (final_hidden_state, final_cell_state) = self.lstm(embedded)
    output = self.fc(final_hidden_state.squeeze(0))
    return output

In [18]:
model = LSTMModel(len(vocab))


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [20]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [21]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
for epoch in range(epochs):
  total_loss = 0
  for batch_x, batch_y in dataloader:
    batch_x, batch_y = batch_x.to(device), batch_y.to(device)
    optimizer.zero_grad()
    output = model(batch_x)

    loss = criterion(output, batch_y)
    loss.backward()
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")


Epoch: 1, Loss: 165.7803
Epoch: 2, Loss: 146.3346
Epoch: 3, Loss: 132.8596
Epoch: 4, Loss: 120.3049
Epoch: 5, Loss: 107.8165
Epoch: 6, Loss: 96.5075
Epoch: 7, Loss: 85.7729
Epoch: 8, Loss: 76.0596
Epoch: 9, Loss: 67.2163
Epoch: 10, Loss: 59.1027
Epoch: 11, Loss: 52.2223
Epoch: 12, Loss: 45.1960
Epoch: 13, Loss: 39.5912
Epoch: 14, Loss: 34.7385
Epoch: 15, Loss: 30.5709
Epoch: 16, Loss: 26.8089
Epoch: 17, Loss: 23.6598
Epoch: 18, Loss: 21.0076
Epoch: 19, Loss: 18.5686
Epoch: 20, Loss: 16.5265
Epoch: 21, Loss: 14.8247
Epoch: 22, Loss: 13.4261
Epoch: 23, Loss: 12.4555
Epoch: 24, Loss: 11.2217
Epoch: 25, Loss: 10.4985
Epoch: 26, Loss: 9.8627
Epoch: 27, Loss: 9.1081
Epoch: 28, Loss: 8.5982
Epoch: 29, Loss: 8.0761
Epoch: 30, Loss: 7.5909
Epoch: 31, Loss: 7.2411
Epoch: 32, Loss: 6.8953
Epoch: 33, Loss: 6.6012
Epoch: 34, Loss: 6.2888
Epoch: 35, Loss: 5.9832
Epoch: 36, Loss: 5.9597
Epoch: 37, Loss: 5.8327
Epoch: 38, Loss: 5.5972
Epoch: 39, Loss: 5.4030
Epoch: 40, Loss: 5.2363
Epoch: 41, Loss: 5.

In [26]:
def prediction(model,vocab,text):
  tokenized_text = word_tokenize(text.lower())
  numerical_text = text_to_indces(tokenized_text,vocab)
  padded_text = torch.tensor([0] * (61 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)
  output = model(padded_text)
  value, index = torch.max(output, dim=1)
  print(index)
  return text + " " + list(vocab.keys())[index]


In [27]:
prediction(model, vocab, "The course")

tensor([16])


'The course follows'

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset

from collections import Counter
from nltk.tokenize import word_tokenize
import numpy as np
# --------------------------------------------------
# 1. Load IMDb small subset for safety
# --------------------------------------------------
data = load_dataset("imdb")
train_texts_lines = data["train"]["text"][5000:20000]   # take 5k for Colab safety
train_labels = torch.tensor(data["train"]["label"][5000:20000])
print(train_labels.bincount())


tensor([7500, 7500])
